In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [ ]:
true = pd.read_csv("True_new.csv")

In [ ]:
fake = pd.read_csv('fake_news.csv')


In [19]:
fake.head()

,title,text,subject,date,Unnamed: 4
0,Pakistan Reports Surge in Brucellosis Cases Am...,Pakistan health officials noted a 20% rise in ...,Health,"March 16, 2025",NaN
1,Belgium Trials New Drug for Chronic Fatigue Sy...,Belgium began trials for a new drug targeting ...,Health,"March 15, 2025",NaN
2,Uganda Links Deforestation to Rise in Sleeping...,Ugandan researchers reported a 15% increase in...,Health,"March 14, 2025",NaN
3,Canada Develops Wearable to Detect Early Sepsi...,Canada unveiled a wearable device to detect ea...,Health,"March 13, 2025",NaN
4,Australia Investigates Link Between Bushfires ...,Australian health authorities are studying bus...,Health,"March 12, 2025",NaN


In [20]:
true.head()

,title,text,subject,date
0,Scientists in Geneva Develop New Vaccine for M...,A team in Geneva has developed a new meningiti...,News,"March 16, 2025"
1,WHO Reports 20% Decline in Global Cholera Cases,The World Health Organization announces a 20% ...,News,"March 15, 2025"
2,AI Tool in Boston Detects Early Signs of Alzhe...,A new AI system in Boston can detect Alzheimer...,News,"March 14, 2025"
3,London Researchers Link Noise Pollution to Inc...,A study in London finds that noise pollution i...,News,"March 13, 2025"
4,Stem Cell Therapy Improves Vision in Blind Pat...,Scientists in Tokyo used stem cell therapy to ...,News,"March 12, 2025"


In [21]:
fake[ 'class']=0
true ['class']=1

In [22]:
data = pd.concat([fake,true], axis = 0)

In [23]:
data.sample(10)

,title,text,subject,date,Unnamed: 4,class
352,New Vaccine for Coccidioidomycosis Shows 91% E...,A coccidioidomycosis vaccine in Tucson proves ...,News,"January 7, 2025",NaN,1
226,Poland Investigates Link Between Cement Work a...,Polish studies reported a 20% dermatitis rise ...,Health,"March 14, 2025",NaN,0
3,Canada Develops Wearable to Detect Early Sepsi...,Canada unveiled a wearable device to detect ea...,Health,"March 13, 2025",NaN,0
386,Iraq Links Oil Fields to Heat Syncope,Iraqi research tied oil field work to a 15% he...,Health,"March 16, 2025",NaN,0
115,Obesity Rates Drop in Africa Due to New Health...,The WHO reports a 6% decrease in obesity rates...,News,"February 24, 2025",NaN,1
65,New Wearable in Singapore Tracks Stress Levels...,A wearable device in Singapore monitors stress...,News,"January 10, 2025",NaN,1
335,Barbados Expands Mobile Units for Barmah Fores...,Barbados deployed mobile units to test for Bar...,Health,"March 11, 2025",NaN,0
418,Papua New Guinea Expands Mobile Units for Bwam...,Papua New Guinea deployed mobile units to test...,Health,"March 12, 2025",NaN,0
306,AI-Powered Tool Enhances Liver Cancer Detectio...,An AI tool in Valencia improves liver cancer d...,News,"February 22, 2025",NaN,1
56,Poland Links Steel Industry to Lung Infections,Polish research tied steel production to a 15%...,Health,"March 16, 2025",NaN,0


In [24]:
data = data.drop (["title", "subject", "date"], axis = 1)

In [25]:
data.reset_index (inplace=True)

In [26]:
data.drop (['index'],axis = 1, inplace=True)

In [27]:
data.sample(5)

,text,Unnamed: 4,class
316,Cyprus unveiled a wearable to monitor Churg-St...,NaN,0
860,An AI tool in Dublin predicts gout flare-ups w...,NaN,1
111,Switzerland credited better hygiene education ...,NaN,0
231,Indonesian studies reported a 15% schistosomia...,NaN,0
842,Manila reports a 11% decrease in laryngitis ca...,NaN,1


In [28]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*P\]', "", text)
    text = re.sub("\\W", " ", text)
    text = re.sub("https?:://\S+|www.\S+", "'", text)
    text = re.sub("<.*?>+", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub("\n", "", text)
    text = re.sub("\w*\d\w*", "'", text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_49231/913239572.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*P\]', "", text)
/tmp/ipykernel_49231/913239572.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub("https?:://\S+|www.\S+", "'", text)
/tmp/ipykernel_49231/913239572.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub("\w*\d\w*", "'", text)


In [29]:
data["text"] = data["text"].apply(clean_text)

In [30]:
x=data ["text"]
y=data ["class"]
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25, random_state=42)

In [31]:
vectorizer = TfidfVectorizer ()
xv_train = vectorizer. fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [32]:
lr = LogisticRegression()
lr. fit(xv_train, ytrain)

LogisticRegression()

In [33]:
prediction = lr.predict(xv_test)
lr.score(xv_test, ytest)

1.0

In [34]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       155
           1       1.00      1.00      1.00       104

    accuracy                           1.00       259
   macro avg       1.00      1.00      1.00       259
weighted avg       1.00      1.00      1.00       259



In [35]:
joblib. dump(vectorizer, "vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

['lr_model.jb']